# Simulating Complex Physical Phenomena

<div style="background-color: #f0f8ff; border: 2px solid #4682b4; padding: 10px;">
<a href="https://colab.research.google.com/github/DeepTrackAI/DeepLearningCrashCourse/blob/main/Ch011_GNN/ec11_A_dynamics/dynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<strong>If using Colab/Kaggle:</strong> You need to uncomment the code in the cell below this one.
</div>

In [ ]:
# !pip install deeplay  # Uncomment if using Colab/Kaggle.

This notebook provides you with a complete code example to simulate complex systems of interacting particles using graph neural networks.

<div style="background-color: #f0f8ff; border: 2px solid #4682b4; padding: 10px;">
<strong>Note:</strong> This notebook contains the Code Example 11-A from the book  

**Deep Learning Crash Course**  
Benjamin Midtvedt, Jesús Pineda, Henrik Klein Moberg, Harshith Bachimanchi, Joana B. Pereira, Carlo Manzo, Giovanni Volpe  
No Starch Press, San Francisco (CA), 2025  
ISBN-13: 9781718503922  

[https://nostarch.com/deep-learning-crash-course](https://nostarch.com/deep-learning-crash-course)

You can find the other notebooks on the [Deep Learning Crash Course GitHub page](https://github.com/DeepTrackAI/DeepLearningCrashCourse).
</div>

## Working with the SAND Dataset

Download the SAND data ...

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="DeepTrackAI/Sand", local_dir="sand_dataset",
                  allow_patterns=["*.npz", "*.json"], repo_type="dataset")

... load the SAND data ...

In [ ]:
import numpy as np

def load_npz_data(path):
    """Load NPZ data."""
    with np.load(path, allow_pickle=True) as data_file:
        data = [item for _, item in data_file.items()]
    return data

train_data = load_npz_data("sand_dataset/train.npz")
val_data = load_npz_data("sand_dataset/valid.npz")
test_data = load_npz_data("sand_dataset/test.npz")

... load the SAND metadata ...

In [ ]:
import json

with open("sand_dataset/metadata.json", "r") as data_file:
    metadata = json.load(data_file)

... print the metadata ...

In [ ]:
print(json.dumps(metadata, indent=4))

... prepare a video of a SAND simulation ...

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

sample_id = np.random.randint(0, len(train_data))
r = train_data[sample_id][0]  # Particle positions.

fig, ax = plt.subplots(figsize=(6, 6))
scatter = ax.scatter([], [], s=50, c="y", edgecolors="k", linewidth=0.5)
ax.set_xlim(0, 1); ax.set_xticks([]); ax.set_ylim(0, 1); ax.set_yticks([])

def update(frame):
    """Update frame."""
    scatter.set_offsets(r[frame])
    return [scatter]

ani = FuncAnimation(fig, update, frames=len(r), interval=10, blit=True)
video = HTML(ani.to_jshtml())
plt.close()

... and visualize it.

In [ ]:
video

## Building a Graph Network-Based Simulator

Implement the message-passing model ...

In [ ]:
import deeplay as dl

model = dl.GraphToNodeMPM(hidden_features=[64,] * 9, out_features=2)

... incorporate skip connections in the message-passing layer ...

In [ ]:
import torch

rmp_backbone = dl.ResidualMessagePassingNeuralNetwork(
    hidden_features=model.backbone.hidden_features,
    out_features=model.backbone.out_features, out_activation=torch.nn.ReLU,
)
model.replace("backbone", rmp_backbone)
model.build()

... and print the model.

In [ ]:
print(model)

## Building the Dataset

Implement a function to compute the node attributes ...

In [ ]:
def compute_node_attr(r_next, r, n_std, metadata=metadata):
    """Compute node attributes."""
    v = np.diff(r, axis=1)  # Velocities.
    v_mean = np.array(metadata["vel_mean"])
    v_std = np.array(metadata["vel_std"])
    v = (v - v_mean) / (v_std ** 2 + n_std ** 2) ** 0.5
    v = v.reshape(r_next.shape[0], -1)

    boundaries = np.array(metadata["bounds"])
    distance_to_lower_bound = r_next - boundaries[:, 0][None]
    distance_to_upper_bound = boundaries[:, 1][None] - r_next
    distance_to_bounds = np.concatenate(
        [distance_to_lower_bound, distance_to_upper_bound], axis=-1,
    )
    norm_distance_to_bounds = np.clip(
        distance_to_bounds / metadata["default_connectivity_radius"], -1, 1,
    )

    return np.concatenate([v, norm_distance_to_bounds], axis=-1)

... implement a function to compute graph connectivity and edge attributes ...

In [ ]:
def compute_connectivity(r, metadata=metadata):
    """Compute graph connectivity from particle positions and radii."""
    Dr = r[:, None, :] - r[None, :, :]  # Displacements.
    D = np.linalg.norm(Dr, axis=-1)  # Distance matrix.
    radius = metadata["default_connectivity_radius"]
    mask = D < radius
    np.fill_diagonal(mask, False)  # Eliminate self-connections.
    edge_index = np.argwhere(mask).T
    edge_attr = np.concatenate([Dr[mask], D[mask][:, None]], axis=-1) / radius
    return edge_index, edge_attr

... implement a function to compute the graph representation for the positions ...

In [ ]:
def compute_graph(r, n_std):
    """Compute the graph representation for the positions."""
    r_next = r[:, -1]
    node_attr = compute_node_attr(r_next, r, n_std)
    edge_index, edge_attr = compute_connectivity(r_next)
    return node_attr, edge_index, edge_attr

... implement a class to manage the dataset with particle simulations ...

In [ ]:
from torch_geometric.data import Data

class ParticleDataset(torch.utils.data.Dataset):
    """Dataset for particle simulations."""

    def __init__(self, data, metadata, Dt, n_std):
        """Initialize dataset."""
        super().__init__()
        self.data, self.metadata, self.Dt, self.n_std, self.traj_length = \
            data, metadata, Dt, n_std, len(data[0][0])

    def get_r(self, i):
        """Get a position window."""
        sample_id, r_start = divmod(i, self.traj_length - self.Dt)
        r = self.data[sample_id][0].copy()
        r_window = np.transpose(r[r_start:r_start + self.Dt], (1, 0, 2))
        r_next = r[r_start + self.Dt]
        return r_window, r_next

    def noise(self, r_window, n_std):
        """Generate random walk noise to be added to a position window."""
        v = np.diff(r_window, axis=1)
        v_noise = (np.random.randn(*list(v.shape)) * n_std / v.shape[1] ** 0.5)
        noise = np.concatenate([
            np.zeros_like(v_noise[:, 0:1]), np.cumsum(v_noise, axis=1),
        ], axis=1)
        return noise

    def __len__(self):
        """Return the total number of position windows in the dataset."""
        return len(self.data) * (self.traj_length - self.Dt)

    def __getitem__(self, i):
        """Get a position window from the dataset."""
        r_window, r_next = self.get_r(i)

        noise = self.noise(r_window, self.n_std)
        r_window, r_next = r_window + noise, r_next + noise[:, -1]

        v_current = r_window[:, -1] - r_window[:, -2]
        v_next = r_next - r_window[:, -1]

        a = v_next - v_current  # Acceleration.
        a_mean = np.array(self.metadata["acc_mean"])
        a_std = np.array(self.metadata["acc_std"])
        a = (a - a_mean) / (a_std ** 2 + self.n_std ** 2) ** 0.5

        node_attr, edge_index, edge_attr = compute_graph(r_window, self.n_std)

        return Data(x=torch.tensor(node_attr, dtype=torch.float32),
                    edge_index=torch.tensor(edge_index, dtype=torch.long),
                    edge_attr=torch.tensor(edge_attr, dtype=torch.float32),
                    y=torch.tensor(a, dtype=torch.float32))

... initialize the training, validation, and testing datasets ...

In [ ]:
Dt, n_std = 6, 3e-4

train_set = ParticleDataset(train_data, metadata, Dt, n_std)
val_set = ParticleDataset(val_data, metadata, Dt, n_std)
test_set = ParticleDataset(test_data, metadata, Dt, n_std)

... define the data loaders ...

In [ ]:
from torch_geometric.data import DataLoader

train_loader, val_loader, test_loader = \
    DataLoader(train_set, batch_size=4, shuffle=True, pin_memory=True), \
    DataLoader(val_set, batch_size=4, shuffle=False, pin_memory=True), \
    DataLoader(test_set, batch_size=4, shuffle=False, pin_memory=True)

... and train the model.

In [ ]:
from lightning.pytorch.callbacks import ModelCheckpoint

regressor = dl.Regressor(
    model, loss=torch.nn.MSELoss(), optimizer=dl.Adam(lr=1e-4),
).create()

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss", dirpath="models",
    filename="SAND-GNS-model{epoch:02d}-val_loss{val_loss:.2f}",
    auto_insert_metric_name=False,
)
trainer = dl.Trainer(max_epochs=5, callbacks=[checkpoint_callback])
trainer.fit(regressor, train_loader, val_loader)

## Loading a pretrained model

Implement a function to determine the device to be used to perform the computations ...

In [ ]:
def get_device():
    """Select device where to perform the computations."""
    if torch.cuda.is_available():
        return torch.device("cuda:0")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

... use to select the device ...

In [ ]:
device = get_device()

... print the selected device ...

In [ ]:
print(device)

... and load a pretrained model.

In [ ]:
import os

best_model_path = os.path.join("models", "SAND-GNS-model.ckpt")
best_model = torch.load(best_model_path, map_location=torch.device(device))
regressor.load_state_dict(best_model["state_dict"]);

## Testing the Model

In [ ]:
trainer.test(regressor, test_loader)

## Simulating the System

Implement a function to simulate the system ...

In [ ]:
def simulate(model, r, metadata, Dt, n_std):
    """Simulate the system."""
    model.eval()

    T = r.shape[0]  # Total time steps.
    r_sim = np.transpose(r[:Dt].copy(), (1, 0, 2))  # Simulated positions.
    for _ in range(T - Dt):
        with torch.no_grad():
            node_attr, edge_index, edge_attr = \
                compute_graph(r_sim[:, -Dt:, :], n_std)
            graph = Data(
                x=torch.tensor(node_attr, dtype=torch.float32),
                edge_index=torch.tensor(edge_index, dtype=torch.long),
                edge_attr=torch.tensor(edge_attr, dtype=torch.float32),
            )
            graph = graph.to(model.device)

            a = model(graph)  # Acceleration.
            a_mean = np.array(metadata["acc_mean"])
            a_std = np.array(metadata["acc_std"])
            a = a.cpu().numpy() * (a_std ** 2 + n_std ** 2) ** 0.5 + a_mean

            v = r_sim[:, -1] - r_sim[:, -2]  # Velocity.
            v_next = v + a  # Next velocity.

            r_next = r_sim[:, -1] + v_next  # Next position.
            r_sim = np.concatenate([r_sim, r_next[:, None]], axis=1)
    return r_sim

... implement a function to animate a simulation ...

In [ ]:
def animate(sample_id, regressor, data, metadata, Dt, n_std):
    """Animate simulation."""
    r = data[sample_id][0]  # Ground truth positions.
    r_sim = np.transpose(simulate(regressor, r, metadata, Dt, n_std),
                         (1, 0, 2))  # Simulated positions.

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    scatters = [
        axs[0].scatter([], [], s=50, c="y", edgecolors="k", linewidth=0.5),
        axs[1].scatter([], [], s=50, c="y", edgecolors="k", linewidth=0.5),
    ]
    axs[0].set_title("Ground Truth"); axs[1].set_title("Simulated")
    axs[0].set_xlim(0, 1); axs[0].set_xticks([])
    axs[0].set_ylim(0, 1); axs[0].set_yticks([])
    axs[1].set_xlim(0, 1); axs[1].set_xticks([])
    axs[1].set_ylim(0, 1); axs[1].set_yticks([])

    def update(frame):
        """Update frame."""
        scatters[0].set_offsets(r[frame])
        scatters[1].set_offsets(r_sim[frame])
        return scatters

    ani = FuncAnimation(fig, update, frames=len(r), interval=10, blit=True)
    video = HTML(ani.to_jshtml())
    plt.close()
    return video

... and try this simulation.

In [ ]:
animate(23, regressor, test_data, metadata, Dt, n_std)